## Phase equilibrium calculations with TawnyCALC (1):
# The chemical system: bulk composition and phase equilibrium

## Bulk composition

Suppose we're considering some package of geological material (the "system"). Its nature and behaviour depend on its chemical composition - the **bulk composition**. The term "bulk" refers to a property of the the whole system, rather than a property of an individual mineral or fluid phases within the system.

We will look at a representative bulk composition for a mid-ocean ridge basalt (**MORB**), as determined by Sun & McDonough (1989; in: Geol. Soc. London Special Publications, 42, 313-345).

## Phase equilibrium

### The chemical system is composed of phases

**Phase equilibrium modelling** lets us predict which phases (different minerals and fluids) would be present in our system, if it were in thermodynamic equilibrium - an approximation that we're often happy to make. The phases present are a function of bulk composition, temperature, and pressure (strictly, stress, but we will ignore this complication for now). For example, here are some predictions for MORB:

|Pressure (kbar) | Temperature (degC) | Phases in most stable equilibrium
|:-------------- |:------------------ |:---------------------------------
|          0.001 |               1150 | olivine, Ca-rich plagioclase, magnetite, _basaltic-andesitic melt_
|              5 |               1000 | labradoritic plagioclase, clinopyroxene, orthopyroxene, magnetite, ilmenite
|              5 |               1150 | labradoritic plagioclase, clinopyroxene, orthopyroxene, magnetite, _basaltic melt_

Notice that in the two calculations at 1150C, melt is part of the stable assemblage. This is significant because the melt is relatively mobile, and may escape from the package of solid material, which itself may be stationary or else following a tectonically-determined path. If the melt escapes, it will change the bulk composition of our system, and of the surrounding material that it infiltrates, driving changes in the equilibrium phase assemblage. It may also transport significant heat, and modify local stress fields.

### Calculating phase equilibria

For the phase equilibrium calculations above, we used:
- Thermodynamic data for the phases:
    - the "igneous set" of HPx-eos (Holland et al 2018; J Petrol. 59 881-900); 31-10-2020 update. 
    - the Holland & Powell dataset (Holland & Powell 2011; J Metamorph. Geol. 29 333-383), version 6.33.
- The phase equilibrium calculation software THERMOCALC (Powell & Holland 1988; J Metamorph. Geol. 6 173-204).

For more information, see [The HPx-eos and THERMOCALC](http://hpxeosandthermocalc.org).


### TawnyCALC

The TawnyCALC package provides a Python environment for THERMOCALC. Below we demonstrate:
- How to set up the MORB system in TawnyCALC and calculate the phase equilibria from above.
- Extraction of equilibrium thermodynamic properties calculated for MORB.
- What happens to the system if its bulk composition is changed, by addition of an aqueous fluid.

Subsequent tutorials will explore sequential calculations, in which the bulk composition of the system changes progressively via loss and/or gain of phases. Such calculations describe **open-system processes**.

## Setting up a TawnyCALC calculation

### Initiating TawnyCALC and loading thermo data

First, import the necessary packages, including TawnyCALC itself:

In [1]:
import tawnycalc as tc
import os

In [2]:


# !!!!!!!!!!!!!! I also set the following environment variable to tell TawnyCALC where to find THERMOCALC  !!!!!!!!!!!!!!!
# %set_env THERMOCALC_EXECUTABLE tc350beta




THERMOCALC is capable of many types of calculation, controlled by scripts. 
Here we set up a clean "context" for TawnyCALC - a structure for assembling scripts, which it will use to drive THERMOCALC:

In [3]:
context = tc.Context(scripts_dir=None)

Next, load the sources of the thermodynamic data:

In [4]:
context.prefs["dataset"] = 633                  # Holland & Powell dataset, version 6.33
context.script["axfile"] = "ig50NCKFMASHTOCr"   # igneous set of HPx-eos

### Setting a bulk composition

We need to define the MORB bulk composition. This is done in terms of molar proportions of oxides:

In [5]:
# Specify the chemical system, a subset of the oxides recognised by THERMOCALC:

oxidestr = "    SiO2  Al2O3   CaO    MgO   FeOt  K2O    Na2O   TiO2     O  "

# Specify the amounts of these oxides, in moles, for MORB: 

MORB = "  52.97   9.19   12.34  12.84  8.23  0.23  2.64  1.06  0.49  "

# Use tc.wt2mol(oxidestr,<MORB_in_mass_proportions>) and
# tc.mol2wt(oxidestr,<cMORB_in_mole_proportions>) 
# to covert from wt% to mole% or vice versa.

Note, the oxides recognised by THERMOCALC are: H2O, SiO2, Al2O3, CaO, MgO, FeOt, K2O, Na2O, TiO2, O, Cr2O3. It isn't necessary to use all of them (in the MORB composition, we have ignored Cr2O3), but they must be provided in this order. In this list, FeOt = all iron as FeO. Given the reaction 2 FeO + O = Fe2O3, we therefore have 
> mFe2O3 = mO <br>
> mFeO = mFeOt - 2 mO

for molar quantities of the oxide units.

Now to tell TawnyCALC about it. Bulk compositions in TawnyCALC are specified via a structure called *rbi* ("raw bulk info"). Start by telling it which oxides are being used:

In [6]:
rbi_sec = tc.rbi(oxides=oxidestr)

TawnyCALC will complain if an invalid oxide name is provided, or if the order is not what it expects.

Now we provide the values of the oxides in MORB. TawnyCALC doesn't actually expect the bulk composition to be provided as a single vector, as we want to do here. Instead, it expects to build up a bulk composition from a collection of phases - the usefulness of this will be  obvious later. Consequently, the bulk composition is specified via a function called _add_phase_, which looks a little odd:

In [7]:
rbi_sec.add_phase("MORB",  "1", MORB)
rbi_sec

           SiO2   Al2O3  CaO    MgO    FeOt  K2O   Na2O  TiO2  O
MORB  1.0  52.97  9.19   12.34  12.84  8.23  0.23  2.64  1.06  0.49

### Calculating the most stable assemblage

Now we will calculate the most stable assemblage at 5 kbar, 1000ºC, as in the table above.

Here we set some scripts to control the calculation:

In [8]:
# Type of calculation

context.script["pseudosection"] = ""  # Calculation involves a bulk composition
context.script[       "dogmin"] = "0" # Seek the most stable assemblage

# Pressure and temperature (P,T)
context.script[    "diagramPT"] = "2 8 800 1200"   # Pmin, Pmax, Tmin, Tmax; must
                                                    # encompass desired P (kbar) and T (degC)

context.script[        "calcP"] = "5"               # P for calc (kbar)
context.script[        "calcT"] = "1000"            # T for calc (degC)


Another set of scripts relates to the phases and phase assemblages involved in the calculation. This is the most difficult part of the process, requiring petrological understanding, and will be discussed at length in subsequent tutorials.

In [9]:
# Phases and phase assemblages involved in the calculation.

context.script[        "with"] = "pl opx cpx mt ilm ol q g liq"  # Phases to consider:
                                # plagioclase, orthopyroxene, magnetite, spinel, ilmenite
                                # olivine, quartz, melt
context.script[    "inexcess"] = ""          # No phases are *assumed* to be in the assemblage 
context.script[    "varrange"] = "4 6"       # Min_variance, max_variance, of assemblages THERMOCALC 
                                             # should try. This limits the number of combinations 
                                             # of phases it should explore, and hence helps to reduce
                                             # the length of a calculation.
                        
# Load a set of starting guesses for the compositions of the phases 
# from a file. To achieve this, we leverage the `thermocalc_script` class 
# which handles general THERMOCALC scripts, and then pull out the required section.
guessfile="Eg1-bulk-comp-stable-assemblage/xyzguess.txt"
start_guess_data = tc.thermocalc_script(filename=guessfile)
context.script["samecoding"] = start_guess_data["samecoding"]
context.script["xyzguess"] = start_guess_data["xyzguess"]

Finally, we must add the rbi structure created above to the scripts (this must be done after specifying "pseudosection"):

In [10]:
context.script["rbi"] = rbi_sec

Now to run THERMOCALC via TawnyCALC:

In [11]:
results = context.execute()

And here is the onscreen output generated from THERMOCALC, showing the phases in the most stable assemblage, and a summary of their compositions and relative proportions (modes).

In [12]:
context.print_output()

THERMOCALC 3.50 (Free Pascal version)

summary output in the file, "tc-wbykxu-o.txt"
other (eg drawpd) output in the file, "tc-wbykxu-dr.txt"
details of calc results in the file, "tc-wbykxu-ic.txt"
initial tables in the file, "tc-wbykxu-it.txt"
csv format in the file, "tc-wbykxu.csv"
more csv format in the file, "tc-wbykxu2.csv"
(these files may not all be populated yet, depending on the calcs;
 thermocalc should delete empty files at the end of each run)
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

THERMOCALC 3.50 running at 14.54 on Mon 1 Feb,2021
using tc-ds633.txt produced at 20.37 on Fri 23 Jun,2017
with axfile tc-ig50NCKFMASHTOCr.txt and scriptfile tc-wbykxu.txt

reading ax: liq fl pli plc ol ksp mu bi g ep cd opx cpx spn hb ilm ky q ru mt 
            sph sill san H2O an 

with: liq pl ol g opx cpx mt ilm q  (from script)
no phases in excess (from script)

specification of xyz starting guesses of phases
in the scriptfile: liq pl ol g opx cpx mt ilm; not in the s

## Output from TawnyCALC

### Characterising the calculated assemblage via the rbi structure

So the stable assemblage predicted for MORB, at 5 kbar and 1000ºC, is 

plagioclase + orthopyroxene + clinopyroxene + magnetite + ilmenite.

Let's first have a look at what the resultant rbi structure, produced by THERMOCALC to represent the most stable assemblage:

In [13]:
rbi_out = results.rbi
rbi_out

               SiO2      Al2O3     CaO       MgO       FeOt      K2O       Na2O      TiO2      O
pl   0.484734  2.45311   0.773445  0.54689   0.0       0.0       0.015337  0.211218  0.0       0.0
opx  0.170675  1.880776  0.093144  0.073426  1.173208  0.675603  0.0       0.002213  0.006272  0.022022
cpx  0.322417  1.953081  0.057309  0.676504  0.748112  0.392143  0.007385  0.042913  0.014948  0.02496
mt   0.011539  0.0       0.052553  0.0       0.651259  1.486535  0.0       0.0       0.757101  0.190346
ilm  0.010636  0.0       0.0       0.0       0.0       1.1087    0.0       0.0       0.8913    0.1087

Each row represents one of the phases constituting the calculated most-stable assemblage. From the third column onwards, the compositition of the phases is given in molar oxides, normalised to correspond with the formula unit of the phases. For example, in the pyroxenes opx and cpx, with formulae 

  (Ca,Mg,Fe2+,Na,K)$^{M2}$(Mg,Fe$^{2+}$,Al,Fe$^{3+}$,Cr,Ti)$^{M1}$(Si,Al)$_2^{T}$O6 
  
the sum of all cations in the relevant rbi row is 4, while the sum of oxygens is 6. 

The second column of the rbi block represents the modal proportion, or mode, of the phase; this column sums to 1. The mode of phase $i$ is the proportion of atoms in the system comprised of $i$. It can be converted to more convenient units, as shown later in this tutorial.

The block printed above looks very different from our initial rbi input:

In [14]:
rbi_sec

           SiO2   Al2O3  CaO    MgO    FeOt  K2O   Na2O  TiO2  O
MORB  1.0  52.97  9.19   12.34  12.84  8.23  0.23  2.64  1.06  0.49

However, the compositions and modal proportions of phases in _rbi_out_ must combine to give the MORB bulk composition. We can check this conveniently by converting _rbi_out_ into a normalised vector of oxides, using

In [15]:
bc = tc.bulk_composition(rbi_out).molepc_oxides
bc

{'SiO2': 52.97526490887504,
 'Al2O3': 9.190912465747044,
 'CaO': 12.34122518414344,
 'MgO': 12.841281019582624,
 'FeOt': 8.230849215340328,
 'K2O': 0.2300244139580861,
 'Na2O': 2.64026336038733,
 'TiO2': 1.0601263737935231,
 'O': 0.4900530581725886}

which does indeed return the MORB bulk composition.

As we shall see in later tutorials, the rbi structure is a useful way to characterise the system during _open system processes_, in which the chemistry of the system changes over time due to the loss or addition of phases. A very simple example is given at the end of this tutorial.

### Additional thermodynamic output

Which properties can we obtain for our calculated most-stable assemblage? These can be listed with:

In [16]:
results.print_keys()

P
T
bulk_composition
modes
output_tc_ic
rbi
site_fractions
thermodynamic_properties
xyz


Taking *site_fractions* as an example, we can view the output:

In [17]:
results.site_fractions

pl   xK       xNa      xCa
     0.03067  0.42244  0.54689
opx  xMgM1    xFeM1    xAlM1    xFe3M1   xTiM1    xMgM2    xFeM2    xCaM2    xNaM2    xSiT     xAlT
     0.71527  0.16736  0.06706  0.04404  0.00627  0.45794  0.46420  0.07343  0.00443  0.94039  0.05961
cpx  xMgM1    xFeM1    xAlM1    xFe3M1   xTiM1    xMgM2    xFeM2    xCaM2    xNaM2    xKM2     xSiT     xAlT
     0.64887  0.21857  0.06770  0.04992  0.01495  0.09925  0.12365  0.67650  0.08583  0.01477  0.97654  0.02346
mt   xMgT     xFeT     xAlT     xFe3T    xMgM     xFeM     xAlM     xFe3M    xTiM
     0.41606  0.27448  0.03925  0.27021  0.11760  0.41568  0.03293  0.05524  0.37855
ilm  xFe2A    xTiA     xFe3A    xFe2B    xTiB     xFe3B
     0.78522  0.10608  0.10870  0.10608  0.78522  0.10870

access values via a dictionary structure:

In [18]:
results.site_fractions['cpx']['xAlM1']

'0.06770'

and obtain more information about them via:

*!!!!!!!!!!!!!!!!!!!!!!!!!!!!!*

The structure _results.thermodynamic_properties_ contains properties, such as density (rho), that are of interest when integrating phase equilibrium calculations with simulations of Earth processes:

In [19]:
results.thermodynamic_properties

     mode        f           G           H           S        V         rho
pl   0.48473396  0.21845261  -4547.7184  -3752.6157  0.6245   10.26739  2.64380
opx  0.17067477  0.28398840  -3233.4037  -2605.1765  0.4934   6.55740   3.40721
cpx  0.32241672  0.28398840  -3379.4152  -2753.4842  0.4916   6.78014   3.30960
mt   0.01153884  0.40569771  -1996.2844  -1464.2793  0.4179   4.71201   4.28553
ilm  0.01063572  0.56797680  -1414.4591  -1042.3030  0.2923   3.25334   4.69045
sys  -           -           -965.60399  -788.90902  0.13879  2.06758   2.99803

In this data block, the properties of the individual phases are given on a per-formula-unit (p.f.u.) basis, consistent with the compositions in the rbi block. In the final row, 'sys' is the system, comprising the whole mineral assemblage. Its properties - the "bulk" properties, are the sum of the "partial" properties contributed by each phase, weighted by the modes. To form this sum, the properties of each phase must be converted from a p.f.u. basis to a per-atom basis.  Column $f$ gives the per-formula-unit to per-atom conversion factor for the phase, equal to 
(number_atoms_in_system)/(100 * number_atoms_in_formula_unit)

(see !!!!!!!!!!!!!!!!! for more information  !!!!!!!!!!!!!!!!!!!!!!!!.)

In order to work with these concepts a little further, and introduce further classes and functions from TawnyCALC, we will re-cast the modes of phases into percentages by volume (vol%). Use _help(...)_ to find out more about a class or function.

In [20]:
# Use class tc.deconstruct_rbi(rbi_block) to extract rows and columns 
# from an rbi block
assemblage  = tc.deconstruct_rbi(rbi_out).phases
oxides      = tc.deconstruct_rbi(rbi_out).oxides


# For each end-member, extract the volume p.f.u.,
# then convert to a per-atom, mode-weighted value to 
# form a partial volume.

partial_volumes = []

for phase in assemblage:
    volume_pfu = float(results.thermodynamic_properties[phase]["V"])
    f = float(results.thermodynamic_properties[phase]["f"])
    mode = float(results.thermodynamic_properties[phase]["mode"])
    partial_volume = volume_pfu * f * mode
    partial_volumes.append(partial_volume)

volume_percents = [ 100*v/sum(partial_volumes) for v in partial_volumes ]

dict(zip( assemblage, volume_percents ))

{'pl': 52.584539104265275,
 'opx': 15.372302838822968,
 'cpx': 30.025769479018294,
 'mt': 1.0668619821115781,
 'ilm': 0.9505265957818909}

Of course, the sum of _partial_volumes_ is the system volume, as given in the _results.thermodynamic_properties_ block above:

In [21]:
sum(partial_volumes)

2.0675816633548787

The function _tc.partial_property()_ will find the partial volumes for us. It will find $f$ itself, if none is provided, but values from THERMOCALC should be used where available to reduce rounding errors.

In [22]:
volumes = [ float(results.thermodynamic_properties[phase]["V"]) for phase in assemblage ]
f_facs = [ float(results.thermodynamic_properties[phase]["f"]) for phase in assemblage ]

partial_volumes = tc.partial_property(rbi_out,volumes,f_facs).values

volume_percents = [ 100*v/sum(partial_volumes) for v in partial_volumes ]

dict(zip( assemblage, volume_percents ))

{'pl': 52.584495619113504,
 'opx': 15.37230957365391,
 'cpx': 30.02576824688458,
 'mt': 1.0668758051465943,
 'ilm': 0.9505507552014252}

Finally, the mass of a phase or system can be calculated, using molar masses imported via

In [23]:
tc.oxide_data().molar_masses

{'H2O': 18.015,
 'SiO2': 60.08,
 'Al2O3': 101.96,
 'CaO': 56.08,
 'MgO': 40.3,
 'FeO': 71.85,
 'FeOt': 71.85,
 'K2O': 94.2,
 'Na2O': 61.98,
 'TiO2': 79.88,
 'O': 16.0,
 'Cr2O3': 151.99}

So we might also write our modal proportions of phases in wt%, using 

In [24]:
masses = [ sum([ tc.oxide_data().molar_masses[ox] * rbi_out[phase][ox] for ox in oxides ]) for phase in assemblage ]
partial_masses = tc.partial_property(rbi_out,masses,f_facs).values
wt_percents = [ 100*m/sum(partial_masses) for m in partial_masses ]
dict(zip( assemblage, wt_percents ))

{'pl': 46.371274426895134,
 'opx': 17.47035246400657,
 'cpx': 33.14618892232143,
 'mt': 1.5250395205364544,
 'ilm': 1.4871446662404015}

## A change to the bulk composition

Supposing that while our package of rock is at 5 kbar, 1000ºC, it is infiltrated by a large volume of water. We have previously modelled it as being "dry", i.e. H2O-free, so this will change its bulk composition dramatically. (In general, *any* change in bulk composition affects compositions and modes of the stable phases, and may potentially change which phases are present in the stable assemblage, even if we have not added a new chemical component.)

We'll start again with a clean context for the calculations (but the same thermo input data):

In [25]:
context = tc.Context(scripts_dir=None)
context.prefs["dataset"] = 633                  # Holland & Powell dataset, version 6.33
context.script["axfile"] = "ig50NCKFMASHTOCr"   # igneous set of HPx-eos

We must redefine the oxide list to include H2O, which goes in first place. We will construct the initial rbi block by adding an H2O fluid to the stable assemblage at 5 kbar, 1000ºC.

In [26]:
oxidestr = "    H2O   SiO2  Al2O3   CaO    MgO   FeOt  K2O    Na2O   TiO2     O  "

rbi_sec = tc.rbi(oxides=oxidestr)

#                 phase   mode         H2O   SiO2      Al2O3     CaO       MgO       FeO       K2O       Na2O      TiO2      O
# lifted from the previous calc 
rbi_sec.add_phase("pl",   "0.484734", "0.0   2.45311   0.773445  0.54689   0.0       0.0       0.015337  0.211218  0.0       0.0")
rbi_sec.add_phase("opx",  "0.170675", "0.0   1.880776  0.093144  0.073426  1.173208  0.675603  0.0       0.002213  0.006272  0.022022")
rbi_sec.add_phase("cpx",  "0.322417", "0.0   1.953081  0.057309  0.676504  0.748112  0.392143  0.007385  0.042913  0.014948  0.02496")
rbi_sec.add_phase("mt",   "0.011539", "0.0   0.0       0.052553  0.0       0.651259  1.486535  0.0       0.0       0.757101  0.190346")
rbi_sec.add_phase("ilm",  "0.010636", "0.0   0.0       0.0       0.0       0.0       1.1087    0.0       0.0       0.8913    0.1087")

# adding some H2O
rbi_sec.add_phase("H2O",  "0.055",    "1.0   0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0     ")

# inspect the bulk composition in mol% oxides expressed by
# this rbi block: 
bch = tc.bulk_composition(rbi_sec).molepc_oxides
bch

{'H2O': 4.9487918425102375,
 'SiO2': 50.353629320516454,
 'Al2O3': 8.7360733393899,
 'CaO': 11.730483638964731,
 'MgO': 12.205792752011709,
 'FeOt': 7.82352162080225,
 'K2O': 0.2186409845243464,
 'Na2O': 2.509602222587696,
 'TiO2': 1.0076629262869299,
 'O': 0.4658013524057718}

Now for the other scripts:

In [27]:
# Type of calculation
context.script["pseudosection"] = ""  # Calculation involves a bulk composition
context.script[       "dogmin"] = "0" # Seek the most stable assemblage

# Pressure and temperature (P,T)
context.script[    "diagramPT"] = "2 8 800 1200"   # Pmin, Pmax, Tmin, Tmax; must
                                                    # encompass desired P (kbar) and T (degC)
context.script[        "calcP"] = "5"               # P for calc (kbar)
context.script[        "calcT"] = "1000"            # T for calc (degC)
# Phases and phase assemblages involved in the calculation.
#

context.script[        "with"] = "hb bi pl opx cpx mt ilm ol q g liq"  
                     # Phases to consider:
                     # hornblende, biotite,    <-- H2O-bearing phases; couldn't appear in previous calc 
                     # plagioclase, orthopyroxene, magnetite, spinel, ilmenite
                     # olivine, quartz, melt
context.script[    "inexcess"] = ""          # No phases are *assumed* to be in the assemblage 
context.script[    "varrange"] = "6 6"       # Min_variance, max_variance, of assemblages THERMOCALC 
                                             # should try. This limits the number of combinations 
                                             # of phases it should explore, and hence helps to reduce
                                             # the length of a calculation.
                        
# Load a set of starting guesses for the compositions of the phases 
# from a file. To achieve this, we leverage the `thermocalc_script` class 
# which handles general THERMOCALC scripts, and then pull out the required section.
guessfile="Eg1-bulk-comp-stable-assemblage/xyzguess2.txt"
start_guess_data = tc.thermocalc_script(filename=guessfile)
context.script["samecoding"] = start_guess_data["samecoding"]
context.script["xyzguess"] = start_guess_data["xyzguess"]

# Load the rbi script
context.script["rbi"] = rbi_sec

Call THERMOCALC:

In [28]:
results = context.execute(timeout=100)

In [29]:
context.print_output()

THERMOCALC 3.50 (Free Pascal version)

summary output in the file, "tc-rmdpbo-o.txt"
other (eg drawpd) output in the file, "tc-rmdpbo-dr.txt"
details of calc results in the file, "tc-rmdpbo-ic.txt"
initial tables in the file, "tc-rmdpbo-it.txt"
csv format in the file, "tc-rmdpbo.csv"
more csv format in the file, "tc-rmdpbo2.csv"
(these files may not all be populated yet, depending on the calcs;
 thermocalc should delete empty files at the end of each run)
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

THERMOCALC 3.50 running at 14.54 on Mon 1 Feb,2021
using tc-ds633.txt produced at 20.37 on Fri 23 Jun,2017
with axfile tc-ig50NCKFMASHTOCr.txt and scriptfile tc-rmdpbo.txt

reading ax: liq fl pli plc ol ksp mu bi g ep cd opx cpx spn hb ilm ky q ru mt 
            sph sill san H2O an 

with: liq pl ol bi g opx cpx mt hb ilm q  (from script)
no phases in excess (from script)

specification of xyz starting guesses of phases
in the scriptfile: liq pl ol bi g opx cpx mt hb ilm; 

In [30]:
rbi_out = results.rbi
rbi_out

               H2O       SiO2      Al2O3     CaO       MgO       FeOt      K2O       Na2O      TiO2      O
liq  0.34218   0.353234  1.259328  0.204557  0.131209  0.152431  0.175143  0.015219  0.113953  0.020853  0.007986
pl   0.25465   0.0       2.235018  0.882491  0.764982  0.0       0.0       0.001001  0.116508  0.0       0.0
opx  0.115445  0.0       1.882739  0.089811  0.078585  1.227566  0.621962  0.0       0.00159   0.006345  0.022695
cpx  0.242759  0.0       1.938436  0.059089  0.735481  0.786156  0.344756  0.000973  0.031659  0.011729  0.023379
hb   0.034246  0.816219  5.682433  1.508634  1.502148  2.902042  1.505878  0.005899  0.429614  0.367561  0.212309
ilm  0.01072   0.0       0.0       0.0       0.0       0.0       1.118087  0.0       0.0       0.881913  0.118087

Compare the new stable assemblage with that in the dry rock:

>  pl  opx  cpx  mt  ilm.

We see that:
- The amphibole hornblende (hb) has now joined plagioclase and the two pyroxenes.
- The pure H2O that we added to the rbi block above is not present in the stable assemblage. Instead, the rock has undergone partial melting in response to the addition of the aqueous fluid, and silicate melt (liq) is now abundant (~1/3 per-atom) in the assemblage.

In [31]:
results.thermodynamic_properties

     mode        f           G            H            S        V         rho
liq  0.34217993  0.39355561  -2225.0383   -1718.9701   0.3975   5.58223   2.49463
pl   0.25464974  0.21906211  -4607.7019   -3819.9133   0.6188   10.22074  2.68546
opx  0.11544483  0.28478075  -3249.9432   -2624.4651   0.4913   6.55032   3.38620
cpx  0.24275918  0.28478075  -3403.8453   -2781.8012   0.4886   6.77582   3.29922
hb   0.03424622  0.06865364  -13558.3451  -10959.3144  2.0414   28.00837  3.13926
ilm  0.01072010  0.56956149  -1410.7919   -1038.0433   0.2928   3.25196   4.69474
sys  -           -           -939.33191   -755.28130   0.14456  2.09139   2.85986

Supposing all of the melt now escapes the system (an extreme scenario), how much does this modify the chemistry of the remaining system?

In [32]:
rbi_out['liq']['mode'] = '0.'                       # set melt mode to 0
rbi_out                                             

               H2O       SiO2      Al2O3     CaO       MgO       FeOt      K2O       Na2O      TiO2      O
liq  0.        0.353234  1.259328  0.204557  0.131209  0.152431  0.175143  0.015219  0.113953  0.020853  0.007986
pl   0.25465   0.0       2.235018  0.882491  0.764982  0.0       0.0       0.001001  0.116508  0.0       0.0
opx  0.115445  0.0       1.882739  0.089811  0.078585  1.227566  0.621962  0.0       0.00159   0.006345  0.022695
cpx  0.242759  0.0       1.938436  0.059089  0.735481  0.786156  0.344756  0.000973  0.031659  0.011729  0.023379
hb   0.034246  0.816219  5.682433  1.508634  1.502148  2.902042  1.505878  0.005899  0.429614  0.367561  0.212309
ilm  0.01072   0.0       0.0       0.0       0.0       0.0       1.118087  0.0       0.0       0.881913  0.118087

In [33]:
# Compare the bulk composition in mole% oxides before and after melt loss

# calculate new system bulk comp in mole% oxides
bcn = tc.bulk_composition(rbi_out).molepc_oxides     
                                                     

# format for printing
bn = ["after melt loss"]+[ "{:0.2f}".format(bcn[ox]) for ox in oxides ]
bh = ["before melt loss"]+[ "{:0.2f}".format(bch[ox]) for ox in oxides ]
col = ["oxides"] + tc.deconstruct_rbi(rbi_out).oxides
from tabulate import tabulate
bctable = tabulate([col,bh,bn],tablefmt="plain")
print(bctable)

oxides            H2O    SiO2  Al2O3  CaO    MgO   FeOt  K2O   Na2O  TiO2  O
before melt loss  50.35  8.74  11.73  12.21  7.82  0.22  2.51  1.01  0.47
after melt loss   49.68  8.90  14.82  15.10  8.13  0.02  1.45  1.08  0.53


As expected, melt loss results in the extensive depletion of incompatible oxides (e.g. H2O, K2O) in the system, while having little effect on the compatible oxides.